In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

Limpieza_Clicks = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Clicks.ipynb'))
Limpieza_Auctions = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Auctions.ipynb'))
Limpieza_Installs = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Installs.ipynb'))
Limpieza_Events = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Events.ipynb'))

In [3]:
%run $Limpieza_Clicks
%run $Limpieza_Auctions
%run $Limpieza_Installs
%run $Limpieza_Events

# Cálculo del St

Calculamos el tiempo hasta que un dispositivo vuelva a aparecer en una subasta RTB como la fecha de la primera aparición en la ventana menos la fecha de inicio de la ventana.

In [303]:
training_set_subastas = auctions
training_set_subastas.sort_values(ascending=True,by='date',inplace=True)

In [304]:
#Separamos el set de datos de subastas en "ventanas" de 3 días cada una.
ventana1_subastas = training_set_subastas.loc[(training_set_subastas['date'] > pd.to_datetime('2019-04-18')) & (training_set_subastas['date'] < pd.to_datetime('2019-04-21'))]
ventana2_subastas = training_set_subastas.loc[(training_set_subastas['date'] > pd.to_datetime('2019-04-19')) & (training_set_subastas['date'] < pd.to_datetime('2019-04-22'))]
ventana3_subastas = training_set_subastas.loc[(training_set_subastas['date'] > pd.to_datetime('2019-04-20')) & (training_set_subastas['date'] < pd.to_datetime('2019-04-23'))]
ventana4_subastas = training_set_subastas.loc[(training_set_subastas['date'] > pd.to_datetime('2019-04-21')) & (training_set_subastas['date'] < pd.to_datetime('2019-04-24'))]
ventana5_subastas = training_set_subastas.loc[(training_set_subastas['date'] > pd.to_datetime('2019-04-22')) & (training_set_subastas['date'] < pd.to_datetime('2019-04-25'))]
ventana6_subastas = training_set_subastas.loc[(training_set_subastas['date'] > pd.to_datetime('2019-04-23')) & (training_set_subastas['date'] < pd.to_datetime('2019-04-26'))]
ventana7_subastas = training_set_subastas.loc[(training_set_subastas['date'] > pd.to_datetime('2019-04-24')) & (training_set_subastas['date'] < pd.to_datetime('2019-04-27'))]

In [305]:
fecha_iv1 = pd.to_datetime('2019-04-18')
fecha_iv2 = pd.to_datetime('2019-04-19')
fecha_iv3 = pd.to_datetime('2019-04-20')
fecha_iv4 = pd.to_datetime('2019-04-21')
fecha_iv5 = pd.to_datetime('2019-04-22')
fecha_iv6 = pd.to_datetime('2019-04-23')
fecha_iv7 = pd.to_datetime('2019-04-24')

ventana1_subastas['st'] = (ventana1_subastas.groupby('device_id')['date'].transform(min) -  fecha_iv1).dt.total_seconds()
ventana2_subastas['st'] = (ventana2_subastas.groupby('device_id')['date'].transform(min) -  fecha_iv2).dt.total_seconds()
ventana3_subastas['st'] = (ventana3_subastas.groupby('device_id')['date'].transform(min) -  fecha_iv3).dt.total_seconds()
ventana4_subastas['st'] = (ventana4_subastas.groupby('device_id')['date'].transform(min) -  fecha_iv4).dt.total_seconds()
ventana5_subastas['st'] = (ventana5_subastas.groupby('device_id')['date'].transform(min) -  fecha_iv5).dt.total_seconds()
ventana6_subastas['st'] = (ventana6_subastas.groupby('device_id')['date'].transform(min) -  fecha_iv6).dt.total_seconds()
ventana7_subastas['st'] = (ventana7_subastas.groupby('device_id')['date'].transform(min) -  fecha_iv7).dt.total_seconds()

In [306]:
label1 = ventana1_subastas[['device_id','st']].drop_duplicates()
label2 = ventana2_subastas[['device_id','st']].drop_duplicates()
label3 = ventana3_subastas[['device_id','st']].drop_duplicates()
label4 = ventana4_subastas[['device_id','st']].drop_duplicates()
label5 = ventana5_subastas[['device_id','st']].drop_duplicates()

# Cálculo del Sc

De la misma forma que con St, calculamos el tiempo hasta que un dispositivo vuelva a instalar como la fecha de la primera aparición en la ventana menos la fecha de inicio de la ventana.

In [307]:
training_set_instalaciones = installs
training_set_instalaciones.sort_values(ascending=True,by='created',inplace=True)

In [308]:
#Separamos el set de datos de instalaciones en "ventanas" de 3 días cada una.
ventana1_instalaciones = training_set_instalaciones.loc[(training_set_instalaciones['created'] > pd.to_datetime('2019-04-18')) & (training_set_instalaciones['created'] < pd.to_datetime('2019-04-21'))]
ventana2_instalaciones = training_set_instalaciones.loc[(training_set_instalaciones['created'] > pd.to_datetime('2019-04-19')) & (training_set_instalaciones['created'] < pd.to_datetime('2019-04-22'))]
ventana3_instalaciones = training_set_instalaciones.loc[(training_set_instalaciones['created'] > pd.to_datetime('2019-04-20')) & (training_set_instalaciones['created'] < pd.to_datetime('2019-04-23'))]
ventana4_instalaciones = training_set_instalaciones.loc[(training_set_instalaciones['created'] > pd.to_datetime('2019-04-21')) & (training_set_instalaciones['created'] < pd.to_datetime('2019-04-24'))]
ventana5_instalaciones = training_set_instalaciones.loc[(training_set_instalaciones['created'] > pd.to_datetime('2019-04-22')) & (training_set_instalaciones['created'] < pd.to_datetime('2019-04-25'))]
ventana6_instalaciones = training_set_instalaciones.loc[(training_set_instalaciones['created'] > pd.to_datetime('2019-04-23')) & (training_set_instalaciones['created'] < pd.to_datetime('2019-04-26'))]
ventana7_instalaciones = training_set_instalaciones.loc[(training_set_instalaciones['created'] > pd.to_datetime('2019-04-24')) & (training_set_instalaciones['created'] < pd.to_datetime('2019-04-27'))]

In [309]:
ventana1_instalaciones['sc'] = (ventana1_instalaciones.groupby('ref_hash')['created'].transform(min) -  fecha_iv1).dt.total_seconds()
ventana2_instalaciones['sc'] = (ventana2_instalaciones.groupby('ref_hash')['created'].transform(min) -  fecha_iv2).dt.total_seconds()
ventana3_instalaciones['sc'] = (ventana3_instalaciones.groupby('ref_hash')['created'].transform(min) -  fecha_iv3).dt.total_seconds()
ventana4_instalaciones['sc'] = (ventana4_instalaciones.groupby('ref_hash')['created'].transform(min) -  fecha_iv4).dt.total_seconds()
ventana5_instalaciones['sc'] = (ventana5_instalaciones.groupby('ref_hash')['created'].transform(min) -  fecha_iv5).dt.total_seconds()
ventana6_instalaciones['sc'] = (ventana6_instalaciones.groupby('ref_hash')['created'].transform(min) -  fecha_iv6).dt.total_seconds()
ventana7_instalaciones['sc'] = (ventana7_instalaciones.groupby('ref_hash')['created'].transform(min) -  fecha_iv7).dt.total_seconds()

In [310]:
label1 = pd.merge(label1,ventana1_instalaciones[['ref_hash','sc']].drop_duplicates(),left_on='device_id',right_on='ref_hash',how='outer')  
del label1['ref_hash']
label2 = pd.merge(label2,ventana2_instalaciones[['ref_hash','sc']].drop_duplicates(),left_on='device_id',right_on='ref_hash',how='outer')  
del label2['ref_hash']
label3 = pd.merge(label3,ventana3_instalaciones[['ref_hash','sc']].drop_duplicates(),left_on='device_id',right_on='ref_hash',how='outer')  
del label3['ref_hash']
label4 = pd.merge(label4,ventana4_instalaciones[['ref_hash','sc']].drop_duplicates(),left_on='device_id',right_on='ref_hash',how='outer')  
del label4['ref_hash']
label5 = pd.merge(label5,ventana5_instalaciones[['ref_hash','sc']].drop_duplicates(),left_on='device_id',right_on='ref_hash',how='outer')  
del label5['ref_hash']

tiempo_maximo = 259200
label1.fillna(value=tiempo_maximo,inplace=True)
label2.fillna(value=tiempo_maximo,inplace=True)
label3.fillna(value=tiempo_maximo,inplace=True)
label4.fillna(value=tiempo_maximo,inplace=True)
label5.fillna(value=tiempo_maximo,inplace=True)

labels = [label1,label2,label3,label4,label5]

Los label por ventana quedan de la siguiente forma:

In [311]:
labels[0].head()

,device_id,st,sc
0,1.826644e+18,0.015050,259200.000
1,7.037174e+18,0.029014,259200.000
2,3.392065e+18,0.057540,259200.000
3,1.228982e+18,0.126828,259200.000
4,4.123059e+18,0.132510,17800.664


# FEATURES
Creamos features en base a los datasets. Los features van a estar divididos por dispositivo para poder mergearlos en un unico dataframe con features de todos los dataframes.

### Features de Auctions

In [13]:
auctions_v1 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-18')) & (auctions['date'] < pd.to_datetime('2019-04-21'))]
auctions_v2 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-19')) & (auctions['date'] < pd.to_datetime('2019-04-22'))]
auctions_v3 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-20')) & (auctions['date'] < pd.to_datetime('2019-04-23'))]
auctions_v4 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-21')) & (auctions['date'] < pd.to_datetime('2019-04-24'))]
auctions_v5 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-22')) & (auctions['date'] < pd.to_datetime('2019-04-25'))]

ventanas_auctions = [auctions_v1, auctions_v2, auctions_v3, auctions_v4, auctions_v5]

In [14]:
features_auctions = []

for ventana in ventanas_auctions:
    features_ventana = ventana.groupby('device_id').agg({'device_id':'size'})
    features_ventana.columns = ['cantidad_de_subastas_en_ventana']
    features_auctions.append(features_ventana)

In [15]:
Week_Day = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

for index, ventana in enumerate(ventanas_auctions):
    for dia in Week_Day:
        subastas_por_dia = ventana[ventana['date'].dt.day_name() == dia].groupby('device_id').agg({'device_id':'size'})
        subastas_por_dia.columns = ['cantidad_subastas_{}'.format(dia)]
        
        features_auctions[index] = features_auctions[index].merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
        features_auctions[index].fillna(0, inplace=True)

### Features de Events

In [16]:
events_v1 = events.loc[(events['date'] >= pd.to_datetime('2019-04-18')) & (events['date'] <= pd.to_datetime('2019-04-21'))]
events_v2 = events.loc[(events['date'] >= pd.to_datetime('2019-04-19')) & (events['date'] <= pd.to_datetime('2019-04-22'))]
events_v3 = events.loc[(events['date'] >= pd.to_datetime('2019-04-20')) & (events['date'] <= pd.to_datetime('2019-04-23'))]
events_v4 = events.loc[(events['date'] >= pd.to_datetime('2019-04-21')) & (events['date'] <= pd.to_datetime('2019-04-24'))]
events_v5 = events.loc[(events['date'] >= pd.to_datetime('2019-04-22')) & (events['date'] <= pd.to_datetime('2019-04-25'))]

ventanas_events = [events_v1, events_v2, events_v3, events_v4, events_v5]

In [17]:
features_events = []

for ventana in ventanas_events:
    features_ventana = ventana.groupby('ref_hash').agg({'ref_hash':'size',
                                                        'event_id':lambda x: (x.mode())[0],
                                                        'application_id':lambda x: (x.mode())[0]})
    features_ventana.columns = ['cantidad_de_eventos_en_ventana','evento_mas_frecuente','aplicacion_mas_usada']
    features_events.append(features_ventana)

In [18]:
for index, ventana in enumerate(ventanas_events):
    for dia in Week_Day:
        eventos_por_dia = ventana[ventana['date'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
        eventos_por_dia.columns = ['cantidad_eventos_{}'.format(dia)]
        
        features_events[index] = features_events[index].merge(eventos_por_dia, how='outer', left_index=True, right_index=True)
        features_events[index].fillna(0, inplace=True)

#### Eventos por hora

In [19]:
for index, ventana in enumerate(ventanas_events):
    for hora in range(24):
        eventos_por_hora = ventana[ventana['date'].dt.hour == hora].groupby('ref_hash').agg({'ref_hash':'size'})
        eventos_por_hora.columns = ['cantidad_eventos_hora_{}'.format(hora)]
        features_events[index] = features_events[index].merge(eventos_por_hora, how='outer', left_index=True, right_index=True) 
        features_events[index]['cantidad_eventos_hora_{}'.format(hora)].fillna(0, inplace=True)

### Agrego la version más tipica de SO registrada en los eventos para cada dispositivo

In [154]:
for index,ventana in enumerate(ventanas_events):
    ventana_temp = ventana
    ventana_temp['device_os_version'] = ventana_temp['device_os_version'].fillna(value=0)
    features_events[index] = features_events[index].merge(ventana_temp.groupby('ref_hash').\
                                              agg({'device_os_version':lambda x: (x.mode())[0]}),
                                      left_index=True,
                                      right_index=True,
                                      how='outer')

### Clicks

In [20]:
#Hay clicks que suceden fuera del período de análisis.
clicks_a_considerar = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-18')) & (clicks['created'] < pd.to_datetime('2019-04-27'))]

In [21]:
clicks_v1 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-18')) & (clicks['created'] < pd.to_datetime('2019-04-21'))]
clicks_v2 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-19')) & (clicks['created'] < pd.to_datetime('2019-04-22'))]
clicks_v3 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-20')) & (clicks['created'] < pd.to_datetime('2019-04-23'))]
clicks_v4 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-21')) & (clicks['created'] < pd.to_datetime('2019-04-24'))]
clicks_v5 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-22')) & (clicks['created'] < pd.to_datetime('2019-04-25'))]

ventanas_clicks = [clicks_v1, clicks_v2, clicks_v3, clicks_v4, clicks_v5]

In [22]:
features_clicks = []

for ventana in ventanas_clicks:
    features_ventana = ventana.groupby('ref_hash').agg({'ref_hash':'size'})
    features_ventana.columns = ['cantidad_de_clicks_en_ventana']
    features_clicks.append(features_ventana)

In [23]:
for index, ventana in enumerate(ventanas_clicks):
    for dia in Week_Day:
        clicks_por_dia = ventana[ventana['created'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
        clicks_por_dia.columns = ['cantidad_clicks_{}'.format(dia)]
        
        features_clicks[index] = features_clicks[index].merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
        features_clicks[index].fillna(0, inplace=True)

#### Region en pantalla del click

Se divide en una cuadricula de 9 sectores.Por un lado: Superior, subdividido en izquierda, centro y derecha; Centro con mismas subdivisiones e Inferior también con mismas subdivisiones.

In [24]:
for index, ventana in enumerate(ventanas_clicks):
    ventana_temp = ventana[['ref_hash','touchX','touchY']].fillna(value=-1).groupby('ref_hash')\
                        .agg({'touchX':lambda x: (x.mode())[0],
                              'touchY':lambda x: (x.mode())[0]}).reset_index()
    ventana_temp['SUP_D'] = 0
    ventana_temp['SUP_C'] = 0
    ventana_temp['SUP_I'] = 0
    ventana_temp['CENT_D'] = 0
    ventana_temp['CENT_C'] = 0
    ventana_temp['CENT_I'] = 0
    ventana_temp['INF_D'] = 0
    ventana_temp['INF_C'] = 0
    ventana_temp['INF_I'] = 0
    ventana_temp.loc[(ventana_temp['touchX']>=0.66)&(ventana_temp['touchY']>=10.66),'SUP_D'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.66)&(ventana_temp['touchX']>=0.33)&(ventana_temp['touchY']>=10.66),'SUP_C'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.33)&(ventana_temp['touchX']>=0)&(ventana_temp['touchY']>=10.66),'SUP_I'] = 1
    ventana_temp.loc[(ventana_temp['touchX']>=0.66)&(ventana_temp['touchY']>=5.33)&(ventana_temp['touchY']<10.66),'CENT_D'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.66)&(ventana_temp['touchX']>=0.33)&(ventana_temp['touchY']>=5.33)&(ventana_temp['touchY']<10.66),'CENT_C'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.33)&(ventana_temp['touchX']>=0)&(ventana_temp['touchY']>=5.33)&(ventana_temp['touchY']<10.66),'CENT_I'] = 1
    ventana_temp.loc[(ventana_temp['touchX']>=0.66)&(ventana_temp['touchY']>=0)&(ventana_temp['touchY']<5.33),'INF_D'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.66)&(ventana_temp['touchX']>=0.33)&(ventana_temp['touchY']>=0)&(ventana_temp['touchY']<5.33),'INF_C'] = 1
    ventana_temp.loc[(ventana_temp['touchX']<0.33)&(ventana_temp['touchX']>=0)&(ventana_temp['touchY']>=0)&(ventana_temp['touchY']<5.33),'INF_I'] = 1
    
    #Me quedo con la region mas común para cada dispositivo
    #mergeo con features de clicks
    features_clicks[index] = pd.merge(features_clicks[index].reset_index(),\
            ventana_temp[['ref_hash','SUP_D','SUP_C','SUP_I','CENT_D','CENT_C','CENT_I','INF_D','INF_C','INF_I']].drop_duplicates(),\
                              on='ref_hash',how='inner')

### Installs

In [25]:
installs_v1 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-18')) & (installs['created'] < pd.to_datetime('2019-04-21'))]
installs_v2 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-19')) & (installs['created'] < pd.to_datetime('2019-04-22'))]
installs_v3 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-20')) & (installs['created'] < pd.to_datetime('2019-04-23'))]
installs_v4 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-21')) & (installs['created'] < pd.to_datetime('2019-04-24'))]
installs_v5 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-22')) & (installs['created'] < pd.to_datetime('2019-04-25'))]

ventanas_installs = [installs_v1, installs_v2, installs_v3, installs_v4, installs_v5]

In [26]:
features_installs = []

for ventana in ventanas_installs:
    features_ventana = ventana.groupby('ref_hash').agg({'ref_hash':'size'})
    features_ventana.columns = ['cantidad_de_instalaciones_en_ventana']
    features_installs.append(features_ventana)

In [27]:
for index, ventana in enumerate(ventanas_installs):
    for dia in Week_Day:
        instalaciones_por_dia = ventana[ventana['created'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
        instalaciones_por_dia.columns = ['cantidad_instalaciones_{}'.format(dia)]
        
        features_installs[index] = features_installs[index].merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
        features_installs[index].fillna(0, inplace=True)

### Agrego relacion cant_instalacions/cant_eventos por dispositivo

In [155]:
for i in range(0,5):
    feature_porc = pd.merge(ventanas_events[i].groupby('ref_hash').size().reset_index(name='cantidad_eventos_x_dev'),
                       ventanas_installs[i].groupby('ref_hash').size().reset_index(name='cantidad_inst_x_dev'),
                       on='ref_hash',
                       how='inner')
    feature_porc['Porc_I-E'] =  feature_porc['cantidad_inst_x_dev'] / feature_porc['cantidad_eventos_x_dev']
    feature_porc.set_index('ref_hash',inplace=True)
    del feature_porc['cantidad_inst_x_dev'] 
    del feature_porc['cantidad_eventos_x_dev']
    features_installs[i] = features_installs[i].merge(feature_porc, how='outer', left_index=True, right_index=True)
    features_installs[i].fillna(0, inplace=True)
    

## Mergeo todos los features por dispositivo

In [302]:
#Pasamos a float los device_id para poder joinear con los features
for i in range(0,5):
    labels[i] = labels[i].astype(np.float64)

In [294]:
features_por_ventana = []

for i in range(0,5):
    features_por_ventana.append(pd.merge(features_auctions[i].reset_index(),\
                       features_clicks[i].reset_index(),\
                       left_on='device_id',right_on='ref_hash',how='outer'))
    features_por_ventana[i]['device_id'].fillna(features_por_ventana[i]['ref_hash'],inplace=True)
    del features_por_ventana[i]['ref_hash']
    features_por_ventana[i] = pd.merge(features_por_ventana[i],features_installs[i].reset_index(),\
                        left_on='device_id',right_on='ref_hash',how='outer')
    features_por_ventana[i]['device_id'].fillna(features_por_ventana[i]['ref_hash'],inplace=True)
    del features_por_ventana[i]['ref_hash']
    features_por_ventana[i] = pd.merge(features_por_ventana[i],features_events[i].reset_index(),\
                                       left_on='device_id',right_on='ref_hash',how='outer')
    features_por_ventana[i]['device_id'].fillna(features_por_ventana[i]['ref_hash'],inplace=True)
    del features_por_ventana[i]['ref_hash']
    features_por_ventana[i].fillna(0,inplace=True)
    features_por_ventana[i] = features_por_ventana[i].groupby('device_id').sum().reset_index()
    features_por_ventana[i] = pd.merge(labels[i][['device_id']],features_por_ventana[i],on='device_id',how='inner')
    features_por_ventana[i].sort_values(by='device_id',inplace=True)

# LABEL

In [300]:
labels_b = []

for i in range(0,5):
    labels[i] = (labels[i].loc[labels[i]['device_id'].isin(features_por_ventana[i]['device_id'])])[['device_id','st']]
    labels[i]['binary'] = np.where(labels[i]['st']>=tiempo_maximo,False,True)
    labels[i].sort_values(by='device_id',inplace=True)
    labels_b.append(labels[i][['binary','st']].to_records(index=False))

# Se agrega chequeo para que los df_feautures cumplan condiciones en los algoritmos a ser usados

In [301]:
for i in range(0,5):
    if (features_por_ventana[i].shape[0] != labels[i].shape[0]):
        raise AssertionError("Los df_features de cada ventana tienen que tener misma \
                cantidad de dispositivos(mismos device_id)")